In [ ]:
QUESTS = [
    {"Name": "Orange Quest",
     "Type": "Warfare",
     "Requirements": {"orange": 2}},
     "Rewards": {"VP": 4},
    {"Name": "Black Quest",
     "Type": "Skullduggery",
     "Requirements": {"black": 2}},
     "Rewards": {"VP": 4},
]